# Forecasting county-based microbusiness density levels across the 50 US states and District of Columbia

#### D. Duvall, A. Johnson, K. Paeng, C. Silva, & Y. Zhang
#### IMT 574--University of Washington
#### Mike Stepanovic
#### 11 March, 2023


### Linear Model, Decision Tree, and Combined Model Approach from Time Series Analysis

> ##### Code copied and adapted from Chirag Choudhary's Kaggle codebook (Feb 15th, 2023)
> #### https://www.kaggle.com/code/ch124uec/time-series-hybrid-modeling


In this investigation, we will train a series of Linear Regression models across unique counties in the dataset to eventually integrate with a larger forecasting algorithm based on decision tree regression method.

The inspiration behind the logic for this algorithm was derived from a Kaggle online course on time series-based forecasting (see Holbrook in references). In Holbrook's online course, we see method that builds forecasting algorithm via combination of linear regression and decision tree modeling.

With Linear Regression, we address seasonality to de-trend our microbusiness_density data across each unique county (cfips) series. We see the benefit and applicability of this with our first two linear model renditions.

With decision trees, we address autocorrelation cycles to fit given data and then forecast with its integration into a final combined model informed by the respective linear regression and decision tree models.



In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pandas import read_csv, set_option
from pandas.plotting import scatter_matrix
import seaborn as sns
import csv
#!pip install -U kaleido
import kaleido
import plotly.graph_objs as go
from kaleido.scopes.plotly import PlotlyScope

from IPython.display import Image
Image('path/to/image.png')

import sklearn
from sklearn import preprocessing, metrics
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, roc_auc_score, roc_curve, mean_squared_error, mean_absolute_error, r2_score
import statsmodels.formula.api as smf
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor, plot_tree, export_graphviz
from sklearn.preprocessing import KBinsDiscretizer
from xgboost import XGBRegressor

from datetime import datetime
pd.options.display.float_format = '{:.4f}'.format
from tqdm import tqdm
import pickle
import plotly.express as px
import plotly.graph_objects as go
#!pip install forecast
#from forecast import smape
from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.tsa.stattools import pacf
import warnings
warnings.filterwarnings('ignore')

import pydot
import pydotplus
import os

In [ ]:
#from IPython.display import IFrame
#IFrame("https://www.kaggle.com/competitions/godaddy-microbusiness-density-forecasting", width=700, height=350)

In [4]:
census_path = "/Users/alxbj/Downloads/census_starter.csv"
train_path = "/Users/alxbj/Downloads/train.csv"
test_path = "/Users/alxbj/Downloads/test.csv"
sample_path = "/Users/alxbj/Downloads/sample_submission.csv"
revealed_path = "/Users/alxbj/Downloads/revealed_test.csv"

census_df = pd.read_csv(census_path)
census_df = pd.DataFrame(census_df)

sample_df = pd.read_csv(sample_path)
sample_df = pd.DataFrame(sample_df)

# filling the NaNs
numeric_cols = census_df.select_dtypes(include = ['float64', 'int64']).columns
imputer = SimpleImputer(strategy = 'mean').fit(census_df[numeric_cols])
census_df[numeric_cols] = imputer.transform(census_df[numeric_cols])

test_df = pd.read_csv(test_path)
test_df = pd.DataFrame(test_df)

train_df = pd.read_csv(train_path)
train_df = pd.DataFrame(train_df)

revealed_df = pd.read_csv(revealed_path)
revealed_df = pd.DataFrame(revealed_df)
revealed_df

train_df = pd.concat([train_df, revealed_df], ignore_index=True)

In [ ]:
# Read in the data
#census_df = pd.read_csv('census_starter.csv', sep=",",header=0)
#census_df = pd.DataFrame(census_df)
#census_df = census_df.apply(pd.to_numeric, errors='coerce').fillna(census_df)

#sample_df = pd.read_csv('sample_submission.csv', sep=",",header=0)
#sample_df = pd.DataFrame(sample_df)

# filling the NaNs
#numeric_cols = census_df.select_dtypes(include = ['float64', 'int64']).columns
#imputer = SimpleImputer(strategy = 'mean').fit(census_df[numeric_cols])
#census_df[numeric_cols] = imputer.transform(census_df[numeric_cols])

# Read in the data (again)
#test_df = pd.read_csv('test.csv', sep=",",header=0)
#test_df = pd.DataFrame(test_df)

#train_df = pd.read_csv('train.csv', sep=",",header=0)
#train_df = pd.DataFrame(train_df)

#revealed_df = pd.read_csv('revealed_test.csv', sep=",",header=0)
#revealed_df = pd.DataFrame(revealed_df)

#train_df = pd.concat([train_df, revealed_df], ignore_index=True)

In [ ]:
test_df = test_df.merge(train_df[['cfips', 'county', 'state']].drop_duplicates(), how='left')

In [ ]:
test_df.head()

In [ ]:
train_df.head()

Target variable will be 'microbusiness_density'

In [ ]:
target = 'microbusiness_density'
#target = train_df[['microbusiness_density']]

In [ ]:
# merge train and test sets for main df
df = pd.concat([train_df, test_df])

df['row_id'] = df['row_id'].str.split('_', expand=True)[0]
df['first_day_of_month'] = pd.to_datetime(df['first_day_of_month'])
df['state_fips'] = df['cfips'] // 1000
df['county_fips'] = df['cfips'] % 1000

train_df = df.iloc[:len(train_df)]
test_df = df.iloc[len(train_df):]

### A baseline linear regression model

According to Holbrook's "Time series" seminar (see Kaggle.com), linear regression models can be adapted to time series data in flexible manners such
that we may model higher order polynomial regressors to fit data.

For the linear baseline model below, we begin by plotting microbusiness_density
growth for one county over the entire time series (July, 2019-January, 2023).

For each cfips value across the dataset, we must tailor our regression algorithm to fit a unique county's linear regression curve. 

To customize each regression and define its approach to capturing the moving average of our microbusiness_density values across the set, we call a new 'y_pred' to difference each county-based series.

With this first linear regression, we will simply look at regressions across order 1-4 to identify the model with best residual minimization.

In [ ]:
# create rule to randomly draw a bucket featuring a singular cfip (county) 
# series of  microbusiness_density measures across the 4yr data
cfips = np.random.choice(train_df['cfips'])
cfips

In [ ]:
# temporary subset_df contains 41 values of microbusiness data
subset_df = train_df.loc[train_df['cfips']==cfips].set_index(['first_day_of_month'])
subset_df.shape

In [ ]:
subset_df
#note subset_df shows the microbiz density metrics for cfips drawn using random choice--you get new county each time

In [ ]:
fig = px.line(subset_df, y=target, markers=True, 
              title=f"Plot of {target} for county = cfips {cfips}")
fig.show()
#fig.write_image('my_plotly_chart.png')

In [ ]:
#![my_plotly_chart.png](attachment:e70871f2-04bb-4421-a1ee-6d82260be57d.png)

In [ ]:
lowest_resid = float('inf')
lowest_mse = float('inf')
lowest_smape = float('inf')
best_model_order = None

for n in range(3):
    dp = DeterministicProcess(
        index=subset_df.index,
        constant=True,
        order=n+1,
        drop=True,
    )

    X = dp.in_sample()
    y = subset_df[target]

    trend_model = LinearRegression()
    trend_model.fit(X, y)

    y_pred = pd.Series(trend_model.predict(X), index=X.index)

    fig = px.line(subset_df, y=target, markers=True, 
                  title=f"Plot of {target} for county = cfips {cfips} : Linear Regression Model (Order {n+1})")
    
    fig.add_scatter(x=y_pred.index, y=y_pred, line=dict(dash='dash'), name=f"Order {n+1} Trend")
    
    fig.update_layout(showlegend=True)
    fig.show()
    fig.write_image('my_plotly_chart_2.png')
    
    # Compute cumulative residual valuation, mean squared error, and SMAPE
    resid_val = ((y - y_pred)**2).sum()
    mse_val = mean_squared_error(y, y_pred)
    smape_val = np.mean(np.abs((y - y_pred) / (y + y_pred) / 2)) * 100
    
    # Check if current model has lower residual valuation, mean squared error, or SMAPE
    if resid_val < lowest_resid:
        lowest_resid = resid_val
        best_model_order = n+1
        
    if mse_val < lowest_mse:
        lowest_mse = mse_val
        
    if smape_val < lowest_smape:
        lowest_smape = smape_val
        
# Print out results for model with lowest cumulative residual valuation, mean squared error, and SMAPE
print(f"The best model is the one with order {best_model_order}, which has a cumulative residual valuation of {lowest_resid:.2f}, a mean squared error of {lowest_mse:.2f}, and a SMAPE of {lowest_smape:.2f}%.") 

In [ ]:
# ![my_plotly_chart_2.png](attachment:8951506a-465c-44a3-85ba-ee3588a85534.png)

#### Noting best model iterant accuracy in order 3 polynomial fit

In terms of mean squared error and cumulative residuals, we see that the linear model can be improved perhaps with more appropriate time-based treatment of our series; this will be addressed with a seasonality-informed linear model.

### Linear Regression Model--Seasonality Rendition

We follow Holbrook's method from the Kaggle time series course to difference and then use a seasonality hyperparameter in the 'DeterministicProcess' function to better approximate our target value (microbusiness_density)--effectively, this is "de-trending" the series.

For the last baseline linear regression where we indexed solely with respect to the subset_df's 'first_day_of_the_month' feature, we now index the new model with respect to the entire series of microbusiness_density valuations.

This widescale view will help to identify seasonal-based trends with the addition of 'seasonal = True' hyperparameter. 

How much more accurate can linear approximation be with these tunings?

In [ ]:
# Differencing for stationising time series
y = y - y_pred

In [ ]:
fig = px.line(y, markers=True, title=f"Plot of {target} for county = cfips {cfips} De-trended and de-seasoned time series")
fig.update_layout(showlegend=False)
fig.show()
fig.write_image('my_plotly_chart_3.png')

In [ ]:
# ![my_plotly_chart_3.png](attachment:25990d3d-7480-401f-9b3e-d91278985591.png)

In [ ]:
lowest_resid = float('inf')
lowest_mse = float('inf')
lowest_smape = float('inf')
best_model_order = None

for n in range(3):
    dp = DeterministicProcess(
        index=y.index,
        constant=True,
        order=n+1,
        seasonal=True,
    )

    X = dp.in_sample()
    y = subset_df[target]

    seasonality_model = LinearRegression()
    seasonality_model.fit(X, y)

    y_pred = pd.Series(seasonality_model.predict(X), index=X.index)

    fig = px.line(subset_df, y=target, markers=True, 
                  title=f"Plot of {target} for county = cfips {cfips} : Seasonality-adjusted Linear Regression Model (Order {n+1})")
    
    fig.add_scatter(x=y_pred.index, y=y_pred, line=dict(dash='dash'), name=f"Order {n+1} Trend")
    
    fig.update_layout(showlegend=True)
    fig.show()
    fig.write_image('my_plotly_chart_4.png')
    
    # Compute cumulative residual valuation and mean squared error
    resid_val = ((y - y_pred)**2).sum()
    mse_val = mean_squared_error(y, y_pred)
    smape_val = np.mean(np.abs((y - y_pred) / (y + y_pred) / 2)) * 100
    
    # Check if current model has lower residual valuation or mean squared error
    if resid_val < lowest_resid:
        lowest_resid = resid_val
        best_model_order = n+1
        
    if mse_val < lowest_mse:
        lowest_mse = mse_val
        
        
    if smape_val < lowest_smape:
        lowest_smape = smape_val
        
# Print out results for model with lowest cumulative residual valuation and mean squared error
print(f"The best model is the one with order {best_model_order}, which has a cumulative residual valuation of {lowest_resid:.2f}, a mean squared error of {lowest_mse:.2f}, and a SMAPE of {lowest_smape:.2f}%.")

In [ ]:
# ![my_plotly_chart_4.png](attachment:55e57e43-d86d-47e0-826f-0bebc4790f4d.png)

### Decision Tree Regressor Model

Having minimized error across MSE and cumulative residual valuations in the seasonality-adjusted linear model above, we now turn to the decision tree model to see how we can impact accuracy.

In [ ]:
X = dp.in_sample()
X.head()

In [ ]:
fig = px.line(y, markers=True, 
              title=f"Plot of {target} for county = cfips {cfips} De-trended and de-seasoned time series")
fig.update_layout(showlegend=False)
fig.show()
fig.write_image('my_plotly_chart_5.png')

In [ ]:
# ![my_plotly_chart_5.png](attachment:66b5d94e-5113-44f5-bbd1-1a809a1d1853.png)

In [ ]:
y = y - y_pred

In [ ]:
fig = plot_pacf(y, lags=12)
plt.title(f"PACF Plot of {target}")
plt.xlabel("Lag")
plt.ylabel("Partial Autocorrelation")
plt.savefig("pacf_plot.png")
plt.show()

With this model, we will take care to address partial autocorrelations in our microbusiness_density valuations across each month of the calendar year--based on Choudhary's Kaggle Notebook (2023, February 15th), the code block below represents cycles of temporal autocorrelations across microbusines_density values for a particular cfips (county).

For the example county below, it seems months 2, 4, 6, and 9 represent points for which the forecasting model with cfips (6065) needs to reset itself essentially.

In [ ]:
X = X.dropna()
y = y[X.index]

smape_vals = []
for max_depth in range(2, 11):
    cycle_model = DecisionTreeRegressor(max_depth=max_depth)  # max_depth is a hyper-parameter that requires careful tuning
    cycle_model.fit(X, y)
    y_pred = pd.Series(cycle_model.predict(X), index=X.index)

    fig = px.line(y, markers=True, title=f"Plot of {target} for county = cfips {cfips} with Decision Tree Model (max_depth={max_depth})")
    fig.add_scatter(x=y_pred.index, y=y_pred, line=dict(dash='dash'), name="Decision Tree Model Forecast")
    fig.update_layout(showlegend=True)
    fig.show()
    fig.write_image('my_plotly_chart_6.png')

    # Compute SMAPE
    smape_val = 100/len(y) * np.sum(2 * np.abs(y - y_pred) / (np.abs(y) + np.abs(y_pred)))
    smape_vals.append(smape_val)
    print(f"The Decision Tree model with max_depth={max_depth} has a SMAPE of {smape_val:.2f}.")

best_max_depth = np.argmin(smape_vals) + 2  # add 2 to account for range starting at 2
print(f"The best max_depth for the Decision Tree model is {best_max_depth} with a SMAPE of {smape_vals[best_max_depth-2]:.2f}.")


In [ ]:
# ![my_plotly_chart_6.png](attachment:49ce9fa3-7118-4222-8168-eea9f1a7f3f0.png)

In [ ]:
X = pd.DataFrame()
for i in [2, 4, 6, 9]:
    X[f'y_lag_{i}'] = y.shift(i)
    
X = X.dropna()
y = y[X.index]

In [ ]:
cycle_model = DecisionTreeRegressor(max_depth=2)  # max_depth is a hyper-parameter that requires careful tuning
cycle_model.fit(X, y)

In [ ]:
y_pred = pd.Series(cycle_model.predict(X), index=X.index)

In [ ]:
fig = px.line(y, markers=True, title=f"Plot of {target} for county = cfips {cfips} : Decision Tree Model with max_depth =2")
fig.add_scatter(x=y_pred.index, y=y_pred, line=dict(dash='dash'))
fig.update_layout(showlegend=True)
#fig.show()
fig.write_image('my_plotly_chart_7.png')

In [ ]:
# ![my_plotly_chart_7.png](attachment:65cb8291-6c77-47dd-a32e-2441935e07d3.png)

### Decision Tree Reflection

The decision tree produces excellent results in terms of its SMAPE accuracy across the board, becoming increasingly biased yet accurate as 'max_depth' hyperparameter increases. 


To avoid overfitting in this large dataset, we will not increase max_depth beyond 2 in our final combined forecasting model.

### Combined Model (mixing Linear Regressions and Decision Trees)

For the combined model below, we now have a method implemented via Linear Regression and Decision Tree to approximate fairly accurate curves to fit existing trends in microbusiness_density.

Knowing we have appropriated our model via decision tree to address a particular county's seasonal cycle of fluctuations in microbusiness_density, we can now build a larger model to address fluctuations across each geography with the time series.

This is where we now turn to widening our model's capacities for forecasting.

In [ ]:
def predict_county_microbusinesses(cfips_id, ts_data, horizon_len=1, num_lag_features=1, pacf_threshold=0.0):
    dp = DeterministicProcess(
        index=ts_data.index,
        constant=True,
        order=2,
        drop=True,
        seasonal=True
    )
    
    # Trend and sesonality with Linear Model information
    X_trend_train = dp.in_sample()
    y_trend_train = ts_data[target]

    model = LinearRegression()
    model.fit(X_trend_train, y_trend_train)

    y_pred_trend_train = pd.Series(model.predict(X_trend_train), index=X_trend_train.index)

    X_trend_test = dp.out_of_sample(steps=horizon_len)
    y_pred_trend_test = pd.Series(model.predict(X_trend_test), index=X_trend_test.index)

    # Cycles and autocorrelation accounted via Decision Tree for forecasting ability

    y_cycle_train = y_trend_train - y_pred_trend_train  # detrended and deseasoned
    
    # if pacf threshold provided, select lag features based on correlelogram
    if pacf_threshold > 0.0:        
        lag_features = np.where(np.abs(pacf(y, 12))>=pacf_threshold)[0][1:]
    else:
        lag_features = np.arange(1, num_lag_features+1)
    X_cycle_train = pd.concat({f"lag_{i}":ts_data[target].shift(i) for i in lag_features}, axis=1).dropna()
    y_cycle_train = y_cycle_train.loc[X_cycle_train.index]

    y_pred_cycle_test = pd.Series()

    cycle_model = DecisionTreeRegressor(max_depth=None)
    cycle_model.fit(X_cycle_train, y_cycle_train)

    y_pred_cycle_train = pd.Series(cycle_model.predict(X_cycle_train), index=X_cycle_train.index)

    X_rolling_window = ts_data[target].values#.tolist()

    #identify lag resets based on autocorrelation shifts for each unique county
    for i in X_trend_test.index:
        y_pred_cycle_test[i] = cycle_model.predict([X_rolling_window[[-c for c in lag_features]]])[0]
        np.append(X_rolling_window, y_pred_cycle_test[i])

    y_pred_train = y_pred_trend_train.loc[y_pred_cycle_train.index] + y_pred_cycle_train

    y_pred_test = y_pred_trend_test + y_pred_cycle_test

    y_pred_test.name = target
    y_pred_test.index.name = 'first_day_of_month'

    y_pred_test = y_pred_test.reset_index()
    y_pred_test['row_id'] = y_pred_test['first_day_of_month'].dt.date.apply(lambda x: f"{cfips_id}_{x}")
    
    return y_pred_train, y_pred_test

In [ ]:
def plot_cfips_ts(cfips_id, horizon_len=1, pacf_threshold=0.0, num_lag_features=1):
    ts_data = train_df.loc[train_df['cfips']==cfips_id].set_index('first_day_of_month').asfreq('MS')
    y_pred_train, y_pred_test = predict_county_microbusinesses(cfips_id, ts_data, horizon_len, num_lag_features, pacf_threshold)
    fig = px.line(ts_data[target], markers=True, title=f"{ts_data['county'][0]}, {ts_data['state'][0]} - cfips {cfips_id}")
    fig.add_scatter(x=y_pred_train.index, y=y_pred_train, line=dict(dash='dashdot'), name='train predicted')
    fig.add_scatter(x=y_pred_test['first_day_of_month'], y=y_pred_test[target], line=dict(dash='dash'), name='test predicted')
    fig.update_layout(showlegend=True, yaxis_title=target)
    fig.show()
    fig.write_image('my_plotly_chart_8.png')

In [ ]:
cfips_id = np.random.choice(train_df['cfips'].unique())

In [ ]:
plot_cfips_ts(cfips_id, horizon_len=10, pacf_threshold=0.0, num_lag_features=2)

In [ ]:
# ![my_plotly_chart_8.png](attachment:d7ba68d2-831e-4ef3-93ad-2df3a9ce861f.png)

In [ ]:
submission_df = []

for cfips_id in tqdm(test_df['cfips'].unique()):
    ts_data = train_df.loc[train_df['cfips']==cfips_id].set_index('first_day_of_month').asfreq('MS')
    y_pred_train, y_pred_test = predict_county_microbusinesses(cfips_id, ts_data, horizon_len=8, num_lag_features=2)  # predictions for the next eight months
    submission_df.append(y_pred_test)

submission_df = pd.concat(submission_df).drop(columns='first_day_of_month')

In [ ]:
train_df['row_number'] = train_df.reset_index().index
submission_df['row_number'] = submission_df.reset_index().index
merged_df = train_df.merge(submission_df, on='row_number')
merged_df 

In [ ]:
# Calculate SMAPE
def smape(actual, forecast):
    return 100/len(actual) * np.sum(2 * np.abs(forecast - actual) / (np.abs(actual) + np.abs(forecast)))

# Calculate SMAPE and MAE
smape_score = smape(merged_df['microbusiness_density_x'], merged_df['microbusiness_density_y'])
mae_score = np.mean(np.abs(merged_df['microbusiness_density_x'] - merged_df['microbusiness_density_y']))

# Calculate R-squared
r_squared = r2_score(merged_df['microbusiness_density_x'], merged_df['microbusiness_density_y'])

# Calculate AIC
n = len(merged_df)
k = 2  # two parameters: intercept and slope
residuals = merged_df['microbusiness_density_x'] - merged_df['microbusiness_density_y']
squared_errors = residuals ** 2
rss = squared_errors.sum()
aic = n * np.log(rss/n) + 2 * k

print(f'SMAPE: {smape_score:.4f}')
print(f'MAE: {mae_score:.4f}')
print(f'R-squared: {r_squared:.4f}')
print(f'AIC: {aic:.4f}')


In [ ]:
# Define SMAPE function
def smape(actual, predicted):
    """
    Calculate the Symmetric Mean Absolute Percentage Error between two arrays
    """
    return np.mean((np.abs(actual - predicted) * 200 / (np.abs(actual) + np.abs(predicted))))

# Initialize lists to store errors and coefficients
smape_x_list = []
smape_y_list = []
mae_x_list = []
mae_y_list = []
r_squared_x_list = []
r_squared_y_list = []
aic_x_list = []
aic_y_list = []

# Get list of unique cfips values in merged_df
cfips_list = merged_df['cfips'].unique()

# Loop over each cfips value and calculate errors and coefficients
for cfips in cfips_list:
    # Subset merged_df for current cfips value
    df = merged_df.loc[merged_df['cfips'] == cfips]
    
    # Get actual and predicted values for 'microbusiness_density_x'
    actual_x = df['microbusiness_density_x']
    predicted_x = df['microbusiness_density_x']
    
    # Calculate errors for 'microbusiness_density_x'
    smape_x = smape(actual_x, predicted_x)
    smape_x_list.append(smape_x)
    mae_x = mean_absolute_error(actual_x, predicted_x)
    mae_x_list.append(mae_x)
    r_squared_x = r2_score(actual_x, predicted_x)
    r_squared_x_list.append(r_squared_x)
    aic_x = smf.ols(formula=f"microbusiness_density_x ~ first_day_of_month + active", data=df).fit().aic
    #aic_x = smf.ols(formula=f"microbusiness_density_x ~ first_day_of_month + active", data=df).fit().aic
    aic_x_list.append(aic_x)
    
    # Get actual and predicted values for 'microbusiness_density_y'
    actual_y = df['microbusiness_density_y']
    predicted_y = df['active'] * df['microbusiness_density_y']
    
    # Calculate errors for 'microbusiness_density_y'
    smape_y = smape(actual_y, predicted_y)
    smape_y_list.append(smape_y)
    mae_y = mean_absolute_error(actual_y, predicted_y)
    mae_y_list.append(mae_y)
    r_squared_y = r2_score(actual_y, predicted_y)
    r_squared_y_list.append(r_squared_y)
    aic_y = smf.ols(formula=f"microbusiness_density_y ~ first_day_of_month + active", data=df).fit().aic
    aic_y_list.append(aic_y)

# Calculate average errors and coefficients
avg_smape_x = np.mean(smape_x_list)
avg_smape_y = np.mean(smape_y_list)
avg_mae_x = np.mean(mae_x_list)
avg_mae_y = np.mean(mae_y_list)
avg_r_squared_x = np.mean(r_squared_x_list)
avg_r_squared_y = np.mean(r_squared_y_list)
avg_aic_x = np.mean(aic_x_list)
avg_aic_y = np.mean(aic_y_list)

# Print results
print(f"Average SMAPE for microbusiness_density_x: {avg_smape_x}")
print(f"Average SMAPE for microbusiness_density_y: {avg_smape_y}")
print(f"Average MAE for microbusiness_density_x: {avg_mae_x}")
print(f"Average MAE for microbusiness_density_y: {avg_mae_y}")
print(f"Average R-squared for microbusiness_density_x: {r_squared_x}")
print(f"Average R-squared for microbusiness_density_y: {r_squared_y}")
print(f"Average AIC for aic_x: {aic_x}")
print(f"Average AIC for aic_y: {aic_y}")

In [ ]:
df = merged_df.loc[merged_df['cfips'] == cfips]
df

In [ ]:
# Define SMAPE function
def smape(actual, predicted):
    """
    Calculate the Symmetric Mean Absolute Percentage Error between two arrays
    """
    return np.mean((np.abs(actual - predicted) * 200 / (np.abs(actual) + np.abs(predicted))))

# Initialize lists to store errors and coefficients
smape_x_list = []

mae_x_list = []

r_squared_x_list = []

aic_x_list = []


# Get list of unique cfips values in merged_df
cfips_list = merged_df['cfips'].unique()

# Loop over each cfips value and calculate errors and coefficients
for cfips in cfips_list:
    # Subset merged_df for current cfips value
    df = merged_df.loc[merged_df['cfips'] == cfips]
    
    # Get actual and predicted values for 'microbusiness_density_x'
    actual_x = df['microbusiness_density_x']
    predicted_y = df['microbusiness_density_y']
    
    # Calculate errors for 'microbusiness_density_x'
    smape_x = smape(actual_x, predicted_y)
    smape_x_list.append(smape_x)
    mae_x = mean_absolute_error(actual_x, predicted_y)
    mae_x_list.append(mae_x)
    r_squared_x = r2_score(actual_x, predicted_y)
    r_squared_x_list.append(r_squared_x)
    aic_x = smf.ols(formula=f"microbusiness_density_x ~ first_day_of_month + microbusiness_density_y", data=df).fit().aic
    #aic_x = smf.ols(formula=f"microbusiness_density_x ~ first_day_of_month + active", data=df).fit().aic
    aic_x_list.append(aic_x)
    

# Calculate average errors and coefficients
avg_smape_x = np.mean(smape_x_list)

avg_mae_x = np.mean(mae_x_list)

avg_r_squared_x = np.mean(r_squared_x_list)

avg_aic_x = np.mean(aic_x_list)


# Print results
print(f"Average SMAPE for microbusiness_density_x: {avg_smape_x}")

print(f"Average MAE for microbusiness_density_x: {avg_mae_x}")

print(f"Average R-squared for microbusiness_density_x: {r_squared_x}")

print(f"Average AIC for aic_x: {aic_x}")


### Conclusion

With the final combined forecasting model, we have predicted values across all 25,080 rows of data as well as forecasting provisions for 8 months beyond the timeframe of the known datapoints beyond January, 2023.

Next steps in assessing this model might include post-hoc comparison of actual vs. projected microbusiness density for February-October,2023--it would be very interesting to see exactly how accurate our projections become across counties in terms of their cumulative and month-by-month residual rates, MSE, and SMAPE valuations.

Given that certain counties and states inevitably ended up showing better overall accuracy in terms of regression fit and SMAPE valuation, a step that could be taken to improve this model is to split data into logical geographic regions and then merge several decision trees into an updated combined model.

### References

Brownlee, J. (2016, December 2). "What Is Time Series Forecasting?" in Machine Learning Mastery. 
> https://machinelearningmastery.com/time-series-forecasting/

Choudhary, C. (2023, Februarey 15th). "Time series hybrid modeling" from GoDaddy Microbusiness Forecasting [Codebook]. Kaggle.  
> https://kaggle.com/code/ch124uec/time-series-hybrid-modeling

GPreda. (2023). "GoDaddy Data Cleaning and EDA" from GoDaddy Microbusiness Density Forecasting [Codebook]. Kaggle. 
> https://www.kaggle.com/code/gpreda/godaddy-data-cleaning-and-eda#Check-the-cfips-from-census-data

Holbrook, R. (n.d.). "Time series: apply machine learning to real-world forecasting tasks" from Kaggle Learn.
> https://wwww.kaggle.com/learn/time-series

Kudelya, V. (2023). "Simple baseline with EDA and SMAPE behaviour" from GoDaddy Microbusiness Density Forecasting [Codebook]. Kaggle.
> https://www.kaggle.com/code/vitalykudelya/simple-baseline-with-eda-and-smape-behaviour

RCBhatt. (2023). GoDaddy Microbusiness Density Forecasting [Codebook]. Kaggle. 
> https://www.kaggle.com/code/rcbhatt/godaddy-microbusiness-density-forecasting